* Para el Sistema de Recomendacion instalaremos las librerias necesarias para armarlo debidamente

In [1]:
import pandas as pd
from surprise import SVDpp
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from collections import defaultdict

In [3]:
from surprise import SVD

*Traemos la informacion

In [53]:
movies=pd.read_csv('movies_analisis.csv')

reviews=pd.read_csv('reviews.csv')

In [54]:
movies = movies.iloc[:, 1:]
reviews = reviews.iloc[:, 1:]

*Preparamos los datos de las peliculas para que tome unicamente los Id y descripciones 

In [55]:
movies=movies.drop(['show_id','cast','date_added','release_year','rating','duration_int','duration_type','scored'],axis=1)
movies

,title,listed_in,description,Id
0,ricky velez: here's everything,"comedy, stand up",​comedian ricky velez bares it all with his ho...,hs1
1,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r...",hs2
2,the marksman,"action, thriller",a hardened arizona rancher tries to protect an...,hs3
3,gaia,horror,a forest ranger and two survivalists with a cu...,hs4
4,settlers,"science fiction, thriller",mankind's earliest settlers on the martian fro...,hs5
...,...,...,...,...
21543,zodiac,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",ns8803
21544,zombie dumb,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",ns8804
21545,zombieland,"comedies, horror movies",looking to survive in a world taken over by zo...,ns8805
21546,zoom,"children & family movies, comedies","dragged from civilian life, a former superhero...",ns8806


In [56]:
movies = movies.reindex(columns=['Id', 'title', 'listed_in','description'])
movies.head()


,Id,title,listed_in,description
0,hs1,ricky velez: here's everything,"comedy, stand up",​comedian ricky velez bares it all with his ho...
1,hs2,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
2,hs3,the marksman,"action, thriller",a hardened arizona rancher tries to protect an...
3,hs4,gaia,horror,a forest ranger and two survivalists with a cu...
4,hs5,settlers,"science fiction, thriller",mankind's earliest settlers on the martian fro...


* Ordenamos el dataset de los reviews en user,Id,y scored

In [57]:
reviews=reviews.drop(['Fecha'],axis=1)
reviews.head()

,userId,Id,scored
0,1,as680,1.0
1,1,ns2186,4.5
2,1,hs2381,5.0
3,1,ns3663,5.0
4,1,as9500,5.0


Unimos las dos bases para implementar 

In [58]:
df_movies=reviews.merge(movies, on='Id',how='left')
df_movies.head()

,userId,Id,scored,title,listed_in,description
0,1,as680,1.0,the english civil war,"documentary, special interest",a nation divided. it was a time of great bitte...
1,1,ns2186,4.5,latte and the magic waterstone,"children & family movies, comedies",when a greedy bear steals a magic stone to kee...
2,1,hs2381,5.0,la diosa coronada,"crime, drama, latino",la diosa coronada es una historia donde los su...
3,1,ns3663,5.0,"frankenstein’s monster’s monster, frankenstein",comedies,when actor david harbour finds lost footage of...
4,1,as9500,5.0,kept woman,"drama, suspense",jessica and her fiancé finally move out of the...


In [9]:
df_movies_to_model = df_movies[df_movies.columns[:-3]]
df_movies_to_model

,userId,Id,scored
0,1,as680,1.0
1,1,ns2186,4.5
2,1,hs2381,5.0
3,1,ns3663,5.0
4,1,as9500,5.0
...,...,...,...
11024160,124380,ns5272,4.5
11024161,124380,ns5492,2.5
11024162,124380,hs305,3.5
11024163,124380,ns7881,4.5


* Primero, cargamos los datos en un objeto Dataset de Surprise definiendo reader

In [17]:

# Cargar datos
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)


* cargamos el dataset que queremos cargar de reviews

In [30]:
data = Dataset.load_from_file('scoring.csv', reader=reader)

* A continuación, podemos definir el modelo de recomendación que queremos utilizar. En este caso, utilizaremos un modelo de factorización de matrices denominado SVD (Singular Value Decomposition), que es un modelo popular para la recomendación de películas:

In [31]:
# Definir modelo de recomendación
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)


In [32]:
# Entrenar modelo
model.fit(data.build_full_trainset())


* Probando prediccion con datos dados al modelo

In [50]:
model.predict(uid=1, iid='ns3830')

Prediction(uid=1, iid='ns3830', r_ui=None, est=3.5334550373271236, details={'was_impossible': False})

* el modelo tambien se lo puede descargar de esta manera para luego aplicarlo en alguna api no lo haremos en este caso pero es la opcion

In [ ]:
import joblib

# Guardar modelo entrenado
joblib.dump(model, 'modelo_entrenado_review.pkl')

In [71]:
# Buscar la fila del DataFrame que corresponde a la película
movie_row = df_movies[df_movies['title'] == 'silent night']

# Obtener el valor de la columna "movieId" de la fila encontrada
movie_row


,userId,Id,scored,title,listed_in,description
43560,463,hs2,2.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
64185,684,hs2,4.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
126074,1258,hs2,3.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
137246,1383,hs2,4.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
140446,1428,hs2,5.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
...,...,...,...,...,...,...
10928700,123457,hs2,3.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
10943691,123596,hs2,3.5,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
10969821,123876,hs2,3.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."
10991774,124021,hs2,3.0,silent night,"crime, drama, thriller","mark, a low end south london hitman recently r..."


In [73]:
movie_id = movie_row['Id'].values[1]
movie_id

'hs2'

* colocamos en una funcion lo que queremos para pedir segun un usuario y la pelicula dada del dataframe de movies si se la recomienda o no a dicho usuario segun la consigna dado un ratin hardcodeado que pusimos en la funcion y segun el modelo aplicado o la prediccion estimada (model) que procesamos en los pasos anteriores

In [74]:
def recomendacion_pelicula(usuario, pelicula):
    # Cargar los datos de películas y calificaciones
    
    # Obtener el ID de la película
    movie_row = df_movies[df_movies['title'] == pelicula]
    
    movie_id = movie_row['Id'].values[1]
    
    #movie_id = df_movies[df_movies['title'] == pelicula]['Id'].values[1]
    
    # Hacer la predicción con el modelo para el usuario y la película
    prediction = model.predict(usuario, movie_id).est
    
    # Si la predicción es mayor que 3.5, recomendar ver la película
    if prediction >= 3.3:
        return f"Se recomienda ver {pelicula}."
    else:
        return f"no se recomienda ver la {pelicula}."
    
recomendacion_pelicula(usuario=1, pelicula='silent night')

'Se recomienda ver silent night.'